In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [2]:
df = pd.read_csv('C:/Users/Victoria/gt/MLOp/Housing_Prediction/Training.csv')

In [3]:
import pandas as pd
def read_dataframe(filename):
    df = pd.read_csv(filename)
    df = df.drop('Id', axis=1)
    df = df.drop(columns = ['Alley', 'PoolQC', 'Fence', 'MiscFeature'])

    df = df[df.Neighborhood == 'Avondale']
    df = df[(df.SalePrice >= 100000) & (df.SalePrice <= 500000)]

    categorical = ['BedroomAbvGr', 'GarageCars']
    df[categorical].astype(str)

    return df

# df[categorical].astype(str).dtypes


In [4]:
df_train = read_dataframe('C:/Users/Victoria/gt/MLOp/Housing_Prediction/Training.csv')
df_val = read_dataframe('C:/Users/Victoria/gt/MLOp/Housing_Prediction/Validation.csv')

In [5]:
len(df_train), len(df_val)

(288, 49)

In [6]:
df_train['BedroomAbvGr'] = df_train['BedroomAbvGr'].astype(str)
df_train['GarageCars'] = df_train['GarageCars'].astype(str)

df_train['BED_GAR'] = df_train['BedroomAbvGr'] + '_' + df_train['GarageCars']

df_val['BedroomAbvGr'] = df_val['BedroomAbvGr'].astype(str)
df_val['GarageCars'] = df_val['GarageCars'].astype(str)

df_val['BED_GAR'] = df_val['BedroomAbvGr'] + '_' + df_val['GarageCars']


In [ ]:
df_val

In [9]:
categorical = ['BED_GAR']#'BedroomAbvGr', 'GarageCars']
numerical = ['GarageArea']

dv = DictVectorizer()

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [10]:
target = 'SalePrice'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

c:\Users\Victoria\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


36147.195326956426

In [12]:
la = Lasso(alpha=10000000)
la.fit(X_train, y_train)

y_pred = la.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

c:\Users\Victoria\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


25833.66489247413

In [13]:
if not os.path.exists('models'):
    os.makedirs('models')

# original code to save the file
with open('models/lasso.bin', 'wb') as f_out:
    pickle.dump((dv, la), f_out)

In [ ]:
sns.distplot(y_pred, label='prediction')
sns.distplot(y_train, label='actual')
plt.legend()